In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from fuzzywuzzy import fuzz

/Users/blainecowen/opt/anaconda3/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [33]:
# import nltk

# nltk.download('punkt')
# nltk.download('averaged_perceptron_tagger')
# nltk.download('maxent_ne_chunker')
# nltk.download('words')

In [4]:
import nltk

from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree

In [5]:
df = pd.read_csv('all_results.csv')
# drop where scores is blank
df = df.dropna(subset=['Concert Score 1'])
df.head()

/Users/blainecowen/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Contest Date,Event,Region,School,TEA Code,City,Director,Additional Director,Accompanist,Conference,...,Concert Score 3,Concert Final Score,Sight Reading Judge,Sight Reading Judge.1,Sight Reading Judge.2,Sight Reading Score 1,Sight Reading Score 2,Sight Reading Score 3,Sight Reading Final Score,Award
0,04/12/2022,931-Mixed Chorus,1,Amarillo High School,NaN,Amarillo,Larissa Cunningham,Carolyn Terrell,Cristi Dollar,AAAAA,...,1,1,Paula Edwards,Kathryn Zetterstrom,Stan McGill,1,1,1,1,C
1,04/12/2022,931-Mixed Chorus,1,Amarillo High School,188901001,Amarillo,Carolyn Terrell,Larissa Cunningham,Cristi Dollar,AAAAA,...,1,1,Paula Edwards,Kathryn Zetterstrom,Stan McGill,1,1,1,1,C
2,04/12/2022,931-Mixed Chorus,1,Amarillo High School,NaN,Amarillo,Carolyn Terrell,Larissa Cunningham,Sean Vokes,AAAAA,...,1,1,Paula Edwards,Kathryn Zetterstrom,Stan McGill,1,1,1,1,A
3,04/12/2022,931-Mixed Chorus,1,Borger HS,2159994,Amarillo,Anthony Vickery,NaN,Melanie Sharp,AAAA,...,1,2,Paula Edwards,Kathryn Zetterstrom,Stan McGill,1,1,1,1,1
4,04/12/2022,931-Mixed Chorus,1,Bushland High School,188902,Bushland,Judy Harvel,NaN,Stephanie Fauss,AAA,...,1,1,Paula Edwards,Kathryn Zetterstrom,Stan McGill,1,1,1,1,A


In [6]:
# Make sure all the score columns are numeric
df['Concert Score 1'] = pd.to_numeric(df['Concert Score 1'], errors='coerce')
df['Concert Score 2'] = pd.to_numeric(df['Concert Score 2'], errors='coerce')
df['Concert Score 3'] = pd.to_numeric(df['Concert Score 3'], errors='coerce')
df['Sight Reading Score 1'] = pd.to_numeric(df['Sight Reading Score 1'], errors='coerce')
df['Sight Reading Score 2'] = pd.to_numeric(df['Sight Reading Score 2'], errors='coerce')
df['Sight Reading Score 3'] = pd.to_numeric(df['Sight Reading Score 3'], errors='coerce')


In [7]:
# add a column to show how far away each judges score is from the other 2 judges
df['Concert Judge 1 Variance'] = abs(df['Concert Score 1'] - (df['Concert Score 2'] + df['Concert Score 3'])/2)
df['Concert Judge 2 Variance'] = abs(df['Concert Score 2'] - (df['Concert Score 1'] + df['Concert Score 3'])/2)
df['Concert Judge 3 Variance'] = abs(df['Concert Score 3'] - (df['Concert Score 1'] + df['Concert Score 2'])/2)

df['Concert Judge 1 Skew'] = df['Concert Score 1'] - (df['Concert Score 2'] + df['Concert Score 3'])/2
df['Concert Judge 2 Skew'] = df['Concert Score 2'] - (df['Concert Score 1'] + df['Concert Score 3'])/2
df['Concert Judge 3 Skew'] = df['Concert Score 3'] - (df['Concert Score 1'] + df['Concert Score 2'])/2


# change judge columns to string
df['Concert Judge'] = df['Concert Judge'].astype(str)
df['Concert Judge.1'] = df['Concert Judge.1'].astype(str)
df['Concert Judge.2'] = df['Concert Judge.2'].astype(str)


# fix judge names with / or &
for index, row in df.iterrows():
    # check is concert judge 1 has a / or &
    if '/' in row['Concert Judge']:
        # split on / and take the first name
        judge1= row['Concert Judge'].split('/')[0]
        judge2= row['Concert Judge'].split('/')[1]
        df.at[index, 'Concert Judge'] = judge1
        df.at[index, 'Concert Judge.1'] = judge2
    if '&' in row['Concert Judge']:
        # split on / and take the first name
        judge1= row['Concert Judge'].split('&')[0]
        judge2= row['Concert Judge'].split('&')[1]
        df.at[index, 'Concert Judge'] = judge1
        df.at[index, 'Concert Judge.1'] = judge2

    if '/' in row['Concert Judge.1']:
        # split on / and take the first name
        judge1= row['Concert Judge.1'].split('/')[0]
        judge2= row['Concert Judge.1'].split('/')[1]
        df.at[index, 'Concert Judge.1'] = judge1
        df.at[index, 'Concert Judge.2'] = judge2


    





In [8]:
# make a new df with judge 1, judge 2, and judge 3 all in the same column called judge
df2 = pd.DataFrame()
df2['Concert Judge'] = df['Concert Judge']
df2['Skew'] = df['Concert Judge 1 Skew']
df2['Variance'] = df['Concert Judge 1 Variance']

# append judge 2 and judge 3 to the df
df2 = df2.append(pd.DataFrame({'Concert Judge': df['Concert Judge.1'], 'Skew': df['Concert Judge 2 Skew'], 'Variance': df['Concert Judge 2 Variance']}))
df2 = df2.append(pd.DataFrame({'Concert Judge': df['Concert Judge.2'], 'Skew': df['Concert Judge 3 Skew'], 'Variance': df['Concert Judge 3 Variance']}))


# drop any rows where judge is blank
df2 = df2.dropna(subset=['Concert Judge'])

# change all judge names to lower
df2['Concert Judge'] = df2['Concert Judge'].str.lower()

# remove and 'dr.'
df2['Concert Judge'] = df2['Concert Judge'].str.replace('dr. ','')


# remove any leading or trailing spaces
df2['Concert Judge'] = df2['Concert Judge'].str.strip()

# remove anything in parenthesis
df2['Concert Judge'] = df2['Concert Judge'].str.replace(r"\(.*\)","")

df2['Concert Judge'] = df2['Concert Judge'].str.replace('sally schoptt','sally schott')
df2['Concert Judge'] = df2['Concert Judge'].str.replace('sally shchott','sally schott')

# remove any 'x' at the beginning of a name
df2['Concert Judge'] = df2['Concert Judge'].str.replace(r"^x ","")


# remove any ' - ' and everything after it
df2['Concert Judge'] = df2['Concert Judge'].str.replace(r" - .*","")

# remove periods
df2['Concert Judge'] = df2['Concert Judge'].str.replace('.','')

# remove double spaces
df2['Concert Judge'] = df2['Concert Judge'].str.replace('  ',' ')

# remove anything in apostrophes
df2['Concert Judge'] = df2['Concert Judge'].str.replace(r"'.*","")

# change everything to proper case
df2['Concert Judge'] = df2['Concert Judge'].str.title()

# only return the first 2 words of a name
df2['Concert Judge'] = df2['Concert Judge'].str.split(' ').str[0:2].str.join(' ')

# remove any judge names under 3 characters
df2 = df2[df2['Concert Judge'].str.len() > 3]

# multiply column by 100 to get a percentage
df2['Skew'] = df2['Skew'] * 100

# get rid of any 50s or -50s
df2 = df2[df2['Skew'] != 50]
df2 = df2[df2['Skew'] != -50]
df2 = df2[df2['Variance'] != 50]
df2 = df2[df2['Variance'] != -50]


# check to see if names are reversed
# if sally schott and schott sally are both in the df, replace schott sally with sally schott
for index, row in df2.iterrows():
    try:
        if row['Concert Judge'].split(' ')[1] + ' ' + row['Concert Judge'].split(' ')[0] in df2['Concert Judge'].values:
            # where the name is reversed, replace it with the name in row['Concert Judge']
            df2['Concert Judge'] = df2['Concert Judge'].replace(row['Concert Judge'].split(' ')[1] + ' ' + row['Concert Judge'].split(' ')[0], row['Concert Judge'])
    except:
        pass


        
# drop all rows where Skew is blank
df2 = df2.dropna(subset=['Skew'])

# drop where judge is blank
df2 = df2.dropna(subset=['Concert Judge'])

# compare list to active judges
active_judges = open('active_judges.txt').read().splitlines()

# use nltk to compare judge names to active judges
for index, row in df2.iterrows():
    # for each judge name in df2, compare it to the active judges list
    for judge in active_judges:
        # if the judge name is in the active judges list, replace it with the active judge name
        if fuzz.partial_ratio(row['Concert Judge'], judge) > 80:
            df2.at[index, 'Concert Judge'] = judge
            break


# remove any judges with less than 300 scores
df2 = df2.groupby('Concert Judge').filter(lambda x: len(x) > 300)


# make a csv
df2.to_csv('Judge_Data.csv', index=False)





df2.head()

/var/folders/vd/8wjp31pd1xq0c_f1rjtbl6sr0000gn/T/ipykernel_12872/784717436.py:19: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['Concert Judge'] = df2['Concert Judge'].str.replace('dr. ','')
/var/folders/vd/8wjp31pd1xq0c_f1rjtbl6sr0000gn/T/ipykernel_12872/784717436.py:26: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['Concert Judge'] = df2['Concert Judge'].str.replace(r"\(.*\)","")
/var/folders/vd/8wjp31pd1xq0c_f1rjtbl6sr0000gn/T/ipykernel_12872/784717436.py:32: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['Concert Judge'] = df2['Concert Judge'].str.replace(r"^x ","")
/var/folders/vd/8wjp31pd1xq0c_f1rjtbl6sr0000gn/T/ipykernel_12872/784717436.py:36: FutureWarning: The default value of regex will change from True to False in a future version.
  df2['Concert Judge'] = df2['Concert Judge'].str.replace(r" - .*","")
/var/folders/vd

,Concert Judge,Skew,Variance
91,Mary Wanetta Hill,0.0,0.0
92,Mary Wanetta Hill,0.0,0.0
93,Mary Wanetta Hill,0.0,0.0
94,Mary Wanetta Hill,0.0,0.0
95,Mary Wanetta Hill,0.0,0.0


In [9]:
# group by judge name and get the total variance in one collumn and the average in another
df3 = df2.groupby('Concert Judge').agg({'Skew': ['sum', 'mean'], 'Variance': ['sum', 'mean']})


# add a count column
df3['Count'] = df2.groupby('Concert Judge').size()

# drop the multi index
df3.columns = df3.columns.droplevel(0)

# add column headers 
df3.columns = ['Total Skew', 'Average Skew', 'Total Variance', 'Average Variance', 'Count']

df3.head(100)

# make a csv
df3.to_csv('Judge_Data_Summary.csv', index=True)


